In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

### 1. 소벨 필터

In [2]:
# src = cv2.imread("./fig/bamboo.jpg", cv2.IMREAD_GRAYSCALE)
src = cv2.imread("./fig/plates.png", cv2.IMREAD_GRAYSCALE)


kernel = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], np.float32)
# print(kernel)
# print(kernel.T)

dst1 = cv2.filter2D(src, -1, kernel)
dst2 = cv2.filter2D(src, -1, kernel.T)

cv2.imshow("bamboo", src)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)


while True:
    if cv2.waitKey() == 27:
        break

cv2.destroyAllWindows()

In [ ]:
## Sobel filter
# cv2.Sobel(src, ddepth, dx, dy, dst, ksize, scale, delt, borderType) -> dst
# src : 입력영상
# ddepth : 출력영상의 데이터 타입 (-1)
# dx : x 방향 미분차수
# dy : x 방향 미분차수
# dst : 출력영상
# ksize : 커널의 크기
# scale : 연산결과에 추가적으로 곱할 값
# delta : 연산결과에 추가적으로 더할 값
# borderType : 가장자리 픽셀확장 방식

# magnitude(x, y, magnitude) -> magnitude

In [3]:
src = cv2.imread("./fig/son.jpg", cv2.IMREAD_GRAYSCALE)

dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)
dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)

mag = cv2.magnitude(dx, dy)
mag = np.clip(mag,  0, 255).astype(np.uint8)

ret, dst = cv2.threshold(mag, 150, 255, cv2.THRESH_BINARY)
background = np.zeros((mag.shape[0], mag.shape[1]), np.uint8)
background[mag > 150] = 255

cv2.imshow("src", src)
# cv2.imshow("dx", dx)
# cv2.imshow("dy", dy)
cv2.imshow("mag", mag)
cv2.imshow("dst", dst)
cv2.imshow("background", background)


# plt.imshow(mag, cmap = "gray")
# plt.show()

cv2.waitKey()
cv2.destroyAllWindows()

### 2. Canny 에지필터

In [4]:
## Canny edge
# Canny(image, threshold1, threshold2, edges, apertureSize, L2gradient) -> edges
# image : 입력 영상
# threshold1: 에지결정 하한값
# threshold1: 에지결정 상한값
# edges: None
# apertureSize: 커널사이즈
# L2gradient: gradient 크기 계산, False

In [5]:
src = cv2.imread("./fig/son.jpg", cv2.IMREAD_GRAYSCALE)
dst = cv2.Canny(src, 150, 180)

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

### 3. Hough 변환: 직선검출

In [34]:
## 직선 검출, 곡선 검출; 허프 변환 (Hough transform)
# HoughLinesP(image, rho, theta, threshold, lines, minLineLength, maxLineGap) -> lines
# image: 입력 에지영상
# rho: 축적배열에서 rho의 간격
# theta: 축적배열에서 theta의 간격
# threshold: 직선판단할 임계값
# lines: 선분의 끝좌표 (x1, y1, x2, y2)
# srn = None, stn = None
# minLineLength: 검출한 선분의 최소 길이
# maxLineGap: 직선으로 간주할 최대 에지 점 간격 (끝어진 점을 연결할 기준)

In [6]:
img = cv2.imread("./fig/checkerboard.png", cv2.IMREAD_GRAYSCALE)
edge = cv2.Canny(img, 100, 200)
# ret, edge = cv2.threshold(edge, )

lines = cv2.HoughLinesP(edge, 1, np.pi/360, 100, minLineLength = 10, maxLineGap = 30)
print(lines.shape[0]) # 라인갯수

dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

for i in range(lines.shape[0]):
    pt1 = (lines[i][0][0], lines[i][0][1])
    pt2 = (lines[i][0][2], lines[i][0][3])
    cv2.line(dst, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.imshow('edge', edge)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

52


### 4. Hough 변환: 곡선검출

In [ ]:
# HoughCircles(image, method, dp, minDist[, circles[, param1[, param2[, minRadius[, maxRadius]]]]]) -> circles
# image: 입력영상
# method: cv2.HOUGH_GRADIENT,
# dp: 입력영상에 대한 실제 영상처리 배율, 1,  2 설정
# minDist: 검출된 원들간의 최소거리
# circles: 원좌표 (cx, cy, r), shape = (1, N, 3), dtype = np.float32
# param1: Canny edge max 값
# param2: 축척배열에서 원검출 임계값
# minRadius: 원 크기의 최소값
# maxRadius: 원 크기의 최대값

In [7]:
## Hough transform, 원 검출

src = cv2.imread("./fig/plates.png")
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# src_gray =  255 - src_gray
circles = cv2.HoughCircles(src_gray, cv2.HOUGH_GRADIENT, 1, 
                           50, param1=200, param2= 100, minRadius=50, maxRadius=150)

print(circles.shape[1])

for i in range(circles.shape[1]):
    cx, cy, radius = circles[0][i]
    cv2.circle(src, (int(cx), int(cy)), int(radius), (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow("src-gray", src_gray)

cv2.waitKey()
cv2.destroyAllWindows()

6


### 5. 레이블링 (labeling)

In [ ]:
## 레이블링 (labeling)

# connectedComponentsWithStats(image[, labels[, stats[, centroids[, connectivity[, ltype]]]]]) -> retval, labels, stats, centroids

In [8]:
src = cv2.imread("./fig/symbols.png")
# src = cv2.GaussianBlur(src, (0, 0), 1)

src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 100, 255, cv2.THRESH_BINARY)
# mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, None)


cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
print(stats[1])

for i in range(1, cnts):
    (x, y, w, h, area) = stats[i]

    if area <= 200:
        continue

    cv2.rectangle(src, (x, y, w, h), (0, 0, 255), 2)

print("count = ", cnts)

cv2.imshow("src", src)
# cv2.imshow("gray", src_gray)
cv2.imshow("mask", mask)

cv2.waitKey()
cv2.destroyAllWindows()

[     0      0    512    512 217198]
count =  2


### 6. 외곽선 검출

In [ ]:
## 외곽선 검출
# findContours(image, mode, method[, contours[, hierarchy[, offset]]]) -> contours, hierarchy
    # image: 입력 영상. non-zero 픽셀을 객체로 간주함.
    # mode: 외곽선 검출 모드. cv2.RETR_로 시작하는 상수. 
    #     (cv2.RETR_EXTERNAL, cv2.RETR_LIST,cv2.RETR_CCOMP, cv2.RETR_TREE)
    # method: 외곽선 근사화 방법. cv2.CHAIN_APPROX_로 시작하는 상수.
    # contour: 검출된 외곽선 좌표. numpy.ndarray로 구성된 리스트. 
    # contours[i].shape=(K, 1, 2). contours[i].dtype=numpy.int32.
    # hierarchy: 외곽선 계층 정보. numpy.ndarray. shape=(1, N, 4). dtype=numpy.int32.
    # hierarchy[0, i, 0] ~ hierarchy[0, i, 3]이 순서대로 next, prev, child, parent
    # 외곽선 인덱스를 가리킴. 해당 외곽선이 없으면 -1.
    # offset: 좌표 값 이동 옵셋. 기본값은 (0, 0).
    
# drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]]) -> image

In [12]:
src = cv2.imread("./fig/shape.png")
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(src_gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
contours, hierachy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
for i in range(len(contours)):
    cv2.drawContours(src, contours, i, (0, 0, 255), 1)
    cv2.putText(src, str(i), contours[i][0][0], cv2.FONT_HERSHEY_COMPLEX, 1,
                (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_gray', src_gray)
cv2.imshow("mask", mask)
cv2.waitKey()
cv2.destroyAllWindows()